<a href="https://colab.research.google.com/github/camoragaq/C-uoh/blob/main/Clase9_threads_sincronizaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Threads en C: Sincronización

In [56]:
%%writefile hilo2.c

#include <stdio.h>
#include <string.h>
#include <pthread.h>

int a=0;


void *thread(void *ptr){
  for (int i=0; i<1000; i++){
    a +=1;

  }
  return NULL;
}

int main(){
  pthread_t thread1, thread2;
  

  pthread_create(&thread1, NULL, thread, NULL);
  pthread_create(&thread2, NULL, thread, NULL);

  pthread_join(thread1, NULL);
  pthread_join(thread2, NULL);

  printf("a = %d\n", a);
  return 0;


}

Overwriting hilo2.c


In [57]:
!gcc hilo2.c -o hilo2 -lpthread

In [58]:
!./hilo2

a = 2000


#Ejemplo de la cuenta bancaria

In [44]:
%%writefile banco.c

#include <stdio.h>
#include <pthread.h>

int saldo =100;
pthread_mutex_t m;

void *depositar(void *ptr){
  int *p = (int *) ptr;
  int dinero = *p;

  pthread_mutex_lock(&m);
  saldo+= dinero;
  pthread_mutex_unlock(&m);
  return NULL;

}

void *girar(void *ptr){ //que pasa si tenemos 2 threads quieren retirar al mismo tiempo?
  int *p = (int *) ptr;
  int dinero = *p;

  pthread_mutex_lock(&m);
  if(saldo >= dinero){
    saldo -= dinero;
    return NULL;
  }
  else{
    printf("No hay saldo disponible\n");
  }
  pthread_mutex_unlock(&m);

}


int main(){
  pthread_t thread1, thread2;
  pthread_mutex_init(&m,NULL);

  int giro = 50;
  pthread_create(&thread1, NULL, girar, &giro);
  pthread_create(&thread2, NULL, girar, &giro);

  pthread_join(thread1, NULL);
  pthread_join(thread2, NULL);

  printf("saldo = %d\n", saldo);
  return 0;
}

Overwriting banco.c


In [45]:
!gcc banco.c -o banco -lpthread

In [46]:
!./banco

^C


Implementar el siguiente código del factorial usando mutex.

In [2]:
%%writefile facto.c

#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>

  double mult(int i, int j) {
    int k;
    double p= 1.;
    for (k=i; k<=j; k++)
      p *= k;

    return p;
  }

  typedef struct {
    int i, j;
    pthread_t pid;
    double res;
  } Args;

  void *mult_thread(void *ptr) {
    Args *args= (Args*)ptr;
    args->res= mult(args->i, args->j);
    return NULL;
  }


  int main(int argc, char **argv) {
    int n= atoi(argv[1]);//recibe el numero factorial a la hora de ejecutar el codigo
    int l= (n+1)/2;//divide el numero factorial para que se calcule en ambos threads en partes iguales
    Args args1, args2;//creamos 2 estructuras de tipo Args

    args1.i= 1;
    args1.j= l;
    pthread_create(&args1.pid, NULL, mult_thread, &args1);

    args2.i= l+1;
    args2.j= n;
    pthread_create(&args2.pid, NULL, mult_thread, &args2);

    pthread_join(args1.pid, NULL);
    pthread_join(args2.pid, NULL);
    printf("factorial=%1.14g\n", args1.res*args2.res);
    return 0;
  }

Writing facto.c


El tipo de estructura Args se crea para poder pasar varios argumentos al thread, albergar el pid del thread y retornar el resultado final.

In [3]:
!gcc facto.c -o facto -lpthread

In [5]:
!./facto 5

factorial=120
